In [13]:
#def my_likelihood():
#    return



In [14]:
##Priors Info

#k1 = uniform(1.7*10**-9,1.06*10**-9)##...?
#GA1 = uniform(1.4,3.6)
#GA2 = uniform(0,8)
#GA3 = uniform(0.5,7.5)
#d1 = uniform(229.44,954.72)
#d2 = uniform(d1 + 487.610 , 1652.93)


In [46]:
#Conversion for scaling

MeV = 1 
GeV = 1000*MeV
fm  = 1
c   = 1
metres = (10**15)*fm
km = 1000*metres
second = 2.99*(10**8)*metres
kg = (second**2)/((1.6*(10**(-13)))*((metres)**2))
G = 6.67*10**(-11)*((metres)**3)/(kg*(second)**2)
mbar = 1.66*(10**-27)*kg

import math
dp = 10**-12
mbar = 1.66*(10**-27)*kg
pi = math.pi
Rs = ((2*G*(1.98847*(10**30)*kg))/((c**2)))
K1 =  ((4*pi*(Rs**3))/(1.98847*((10**30)*kg)*(c**2)))*(GeV/(fm)**3)

#importing libraries
import numpy as np
import scipy
import matplotlib.pyplot as plot
from scipy.integrate import odeint
from scipy.stats import uniform

#SFHx
class TOV_Solver:
    def __init__(self,Pc,d1,d2,k1,k2,k3,GA1,GA2,GA3,GA1_R,GA2_R,
                 GA3_R,GA4_R,k1_R,k2_R,k3_R,k4_R,d1_Read,d2_Read,d3_Read):
        self.Pc = Pc
        self.d1 = d1
        self.d2 = d2
        self.k1 = k1
        self.k2 = k2
        self.k3 = k3 
        self.GA1 = GA1
        self.GA2 = GA2
        self.GA3 = GA3
        self.GA1_R = GA1_R
        self.GA2_R = GA2_R
        self.GA3_R = GA3_R
        self.GA4_R = GA4_R
        self.k1_R = k1_R
        self.k2_R = k2_R
        self.k3_R = k3_R
        self.k4_R = k4_R
        self.d1_Read = d1_Read
        self.d2_Read = d2_Read
        self.d3_Read = d3_Read
    
    def EoS_Core(self,d):
        
        condition1 = (d < self.d1)
        condition2 = (d >= self.d1) & (d < self.d2)
        condition3 = (d >= self.d2) 
    
        condition = [condition1, condition2, condition3]
        
        Polytrope1 = lambda d: self.k1*(d)**self.GA1
        Polytrope2 = lambda p: self.k2*(d)**self.GA2
        Polytrope3 = lambda p: self.k3*(d)**self.GA3
        
        function_list = [Polytrope1, Polytrope2, Polytrope3]
        
        p = (np.piecewise(d, condition, function_list))
        return (p)
        
    def EoS_Crust(self,d):
        
        condition1_Read = d < self.d1_Read
        condition2_Read = (d >= self.d1_Read) & (d < self.d2_Read)
        condition3_Read = (d >= self.d2_Read) & (d < self.d3_Read)
        condition4_Read = (d >= self.d3_Read) 
    
        condition_Read = [condition1_Read, condition2_Read, condition3_Read, condition4_Read]
        
        Polytrope1_Read = lambda d: self.k1_R*(d)**(self.GA1_R) 
        Polytrope2_Read = lambda d: self.k2_R*(d)**(self.GA3_R)
        Polytrope3_Read = lambda d: self.k3_R*(d)**(self.GA3_R)
        Polytrope4_Read = lambda d: self.k3_R*(d)**(self.GA4_R)
    
        function_list2 = [Polytrope1_Read, Polytrope2_Read, Polytrope3_Read, Polytrope4_Read]
        
        p = (np.piecewise(d, condition_Read, function_list2))
        return (p)
     
    def EoS_Total(self,d):
        
        #droot = 0
        if d<self.d1_Read:
            droot  = (self.k1/self.k1_R)**(1/(self.GA1_R - self.GA1))
        elif (d< self.d2_Read):
            droot =  (self.k1/self.k2_R)**(1/(self.GA2_R - self.GA1))
        elif (d<self.d3_Read):
            droot =  (self.k1/self.k3_R)**(1/(self.GA3_R - self.GA1))
        else:
            droot =  (self.k1/self.k2_R)**(1/(self.GA2_R - self.GA1))
                                                                  
        condition1 = d < droot
        condition2 = d >= droot
        function1 = self.EoS_Crust(d)
        function2 = self.EoS_Core(d)
        condition_list = [condition1,condition2]
        function_list  = [function1, function2]
        p = (np.piecewise(d, condition_list, function_list))
        return p
                           
                                
    def R0(self,k,GA):
            dc = ((self.Pc/k))**(1/GA) + (self.Pc/(GA - 1))
            P = self.Pc - dp 
            d = ((P/k)**(1/GA) +(P/(GA - 1)))
            nc = ((self.Pc/k)**(1/GA))
            n = (((P)/k)**(1/GA))
            return  (((3/(2*pi*(3*self.Pc + dc)))*((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))))**(0.5)

    def M0(self,k,GA):
            dc = ((self.Pc/k)**(1/GA) + (self.Pc/(GA - 1)))
            P = self.Pc - dp
            d = ((P/k)**(1/GA) +(P/(GA - 1)))
            nc = (((self.Pc)/k)**(1/GA))
            n = (((P)/k)**(1/GA))
            return ((((((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))*(2*dc/(3*self.Pc + dc))))))*self.R0(k,GA)
        
    def f(self,x,d):
        m,r = x
        P = self.EoS_Total(d)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp]  
                       
    def compute_MR(self):
        
        Ro =  self.R0(k = self.k3, GA = self.GA3)
        Mo =  self.M0(k = self.k3, GA = self.GA3)

        R = []
        M = []
        x_i = [Mo,Ro]
        
        P0 = self.Pc - dp
            
        P1 = np.linspace(P0,0,10000)
        solve = odeint(self.f,x_i, P1)
                          
        m = solve[:,0]
        r = solve[:,1]

        M = (m[-1])
        R = (r[-1])*(Rs/km)
        return "M:", M ,"R:", R      

In [47]:
A = TOV_Solver(Pc = 437,
               d1 = 2964053.180994008, 
               d2 = 35818.94244147257, 
               k1 = 1.8250338944950222, 
               k2 = 0.6557614364343343,
               k3 = 84.32007742276306,
               GA1 = 3.85992626, 
               GA2 = 2.62551522,
               GA3 = 1.52897461,
               GA1_R = 1.58425, 
               GA2_R = 1.28733, 
               GA3_R = 0.62223,
               GA4_R = 1.35692, 
               k1_R = 6.157198765836523e-21,
               k2_R =  1.0266961939924408e-25,
               k3_R = 1.0266961939924408e-25,
               k4_R = 1.2350927097424639e-23, 
               d1_Read= 1.3689011050653503e-08,
               d2_Read= 0.00021223873899141752,
               d3_Read= 0.0014740562068772088)

A.compute_MR()

('M:', 3509414.461094946, 'R:', 13017263.089215586)